## FINANCIAL DATA
MODULE 2 | LESSON 4


---

# USING STATISTICAL DISTRIBUTIONS TO MODEL STOCK RETURNS

|  |  |
|:---|:---|
|**Reading Time** |  30 minutes |
|**Prior Knowledge** | Statistics, Cumulative distribution functions |
|**Keywords** | Normal Distribution, Student T distribution, Normality Testing, P-values |

---


*We looked at various ways to measure risk and returns in the last lesson. Here, we take that a step further by using distributions to correctly model risk and returns.*


In [ ]:
import datetime

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import seaborn as sns
import yfinance as yfin
from matplotlib import pyplot as plt
from scipy import stats

yfin.pdr_override()

## 1. How Are Stock Returns Distributed?
Many models and theories surrounding stocks assume a normal distribution. We will try to determine that here with a data-based analysis. Properties of a Gaussian distribution are as follows:

* Mean, median, and mode are all the same
* The data is symmetrical, meaning there are equal counts of observations on both sides of the mean.
* In normally distributed data, 68.25% of all cases fall within +/- one standard deviation from the mean, 95% of all cases fall within +/- two standard deviations from the mean, and 99.7% of all cases fall within +/- three standard deviations from the mean.

Let's start by pulling 20 years of daily price data for the S&P 500. We'll use similar methods we've used in the last few lessons to pull this data and will calculate the log returns here.

One quick way of doing this is to determine how many data points we have on either side of the mean here. We have a bit more than 5,000 data points. The below code takes the count of data points greater than the mean and divides it by the total number of data points. This will give us the percentage of data points greater than the mean.

In [ ]:
# start = datetime.date.today() - datetime.timedelta(365 * 20)
# end = datetime.date.today()
end = datetime.date(2021, 11, 20)
start = end - datetime.timedelta(365 * 20)

prices = pd.DataFrame(web.DataReader(["^GSPC"], start, end)["Adj Close"])

# Rename column to make names more intuitive
prices = prices.rename(columns={"Adj Close": "SP500"})
df = np.log(prices) - np.log(prices.shift(1))
df = df.iloc[1:, 0:]

### 1.1 Are returns symmetric?

One quick way of doing this is to determine how many data points we have on either side of the mean here. We have a bit more then 5,000 data points here. The below code takes the count of data points greater than the mean and divides it by the total data points. This will give us the percentage of data points greater than the mean

In [ ]:
(len(df[df.SP500 > df.SP500.mean()])) / (len(df))

We're getting about 52.6% of data points being greater than the mean, which shows we have a slightly negative skew to this dataset. We can't rule out symmetric returns based on this since it is only a sample of data and is reasonably close to the 50% mark. This makes it hard to say for certain whether S&P 500 returns are symmetric or not, but it is still a reasonable assumption to make here. Also, keep in mind there should be a slight bias towards positive returns anyways, if for no other reason than some drift from inflation.

### 1.2 Is Volatility constant?

In [ ]:
vols = pd.DataFrame(df.SP500.rolling(50).std()).rename(columns={"SP500": "S&P 500 STD"})

# set figure size
plt.figure(figsize=(12, 5))

# plot using rolling average
sns.lineplot(
    x="Date",
    y="S&P 500 STD",
    data=vols,
    label="S&P 500 50 day standard deviation rolling avg",
)
plt.show()

Above, we calculate the rolling standard deviation using a window of 50 days and then make a line chart of it. It can be clearly seen that volatility is anything but constant. This adds another layer of complexity to modeling stock returns, especially the many models which assume constant volatility

## 2. Are Stock Returns Normally Distributed?
The normal distribution is one of the most common distributions used in modeling random variables. Indeed, many phenomena in the natural and social sciences can be modeled by normal distribution. One of the great advantages of the normal distribution is its simplicity. We can completely describe a normal distribution through two numbers: one for the center of the distribution and one for the uncertainty about that center. The first number refers to the mean, and the second number refers to the standard deviation.

Once we have these two numbers, we can draw inferences, estimate percentiles, compute probabilities that a point falls within a region, and more. If our data is well-represented by the normal distribution, then we can confidently use the mean and standard deviation to report our portfolio expected returns and volatilities. If our data is not well represented by the normal distribution, then we need to find other distributions that are more suitable. Thus, when we have a distribution of stock returns, for example, we'll want to start this assessment by visualizing the returns to see if they appear to be normal. Of course, we can follow this up with more quantitative assessments by running statistical tests.

We can visualize the data using the hist() method. We pass in bins = 100 as a parameter to determine the amount of buckets to place the data in. The more bins you have, the more granular the data will look in a histogram. Increasing the bins too much may result in slightly noisy data, which will make it tougher to determine a normal distribution. The chart in Figure 3 looks like it could be normally distributed, but we need to be a little more scientific to determine if that's actually the case or not.

In [ ]:
df.hist(bins=100);

### 2.1 Conducting a normality test

In [ ]:
stats.normaltest((np.array(df.SP500)))

We can use the `normaltest()` method here to determine if the sample data could fit a normal distribution. This method uses D'Agostino and Pearson's normality test, which combines skew and kurtosis to produce an omnibus test of normality.

The null hypothesis of this test is that the sample data fits a normal distribution. Let's assume we want to be 90% confident this data fits a normal distribution. We can compare this to the p-value to see if it's greater than 90%. In this case, the value, 3.61e-252, is extremely small, which leads us to reject the null hypothesis that this data fits a normal distribution.

### 2.2 Testing Skewness and Kurtosis
As one added testing step, we can test the skewness and kurtosis of our distribution using the Jarque-Bera test. The test statistic will always be greater than zero. The further the test statistic is from zero, the more likely the sample data does not match a normal distribution.

Lucky for us, Python has another library for us to use here which really simplifies the analysis. From the `scipy.stats` library, we can use the `jarque_bera()` method directly to our data to get test statistic


In [ ]:
stats.jarque_bera((np.array(df.SP500))).pvalue

The p value of our data here shows once and for all that there is virtually zero chance our sample data is normally distributed<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

### 2.3 Where Does Our Gaussian Distribution Break Down?
So according to the normality test, our data is not normally distributed despite the histogram looking like it may be. So, why is the data failing the normality test? The answer likely comes down to fat tails. Fat tails essentially means that extreme events +/-3 standard deviations away from the mean) are more likely than the normal distribution would imply.

Assuming a normal distribution with a mean of 0.00028 and standard deviation of 0.012, we can determine the probability of any return given that the returns fit a normal distribution.

To determine how many standard deviations away from the mean a specific number is, we need to use

$$\frac{X - \bar{X}}{\text{Sample standard deviation}}$$

 Let's do this for the min and max of the sample data:

In [ ]:
dfMax = df.SP500.max()
dfMin = df.SP500.min()
print(
    "Min return of sample data is %.4f and the maximum return of sample data is %.4f"
    % (dfMin, dfMax)
)

In [ ]:
df.SP500.max()

In [ ]:
df.SP500.min()

In [ ]:
(df.SP500.min() - df.SP500.mean()) / df.SP500.std()

In [ ]:
(df.SP500.max() - df.SP500.mean()) / df.SP500.std()

Over the last 20 years, S&P 500 has had a maximum daily return of 10.96% and a minimum daily return of -12.77%. If we use the formula to determine standard deviations from the mean, we get -10.5 and 8.9 standard deviations away from the mean for the minimum and maximum, respectively. These standard deviations are humongous when compared to the normal distribution. We can see this analytically when we plug in the z score to the `norm.cdf()` method to determine the probability this value could be in a normal distribution:

In [ ]:
stats.norm.cdf(-10.45)

This implies that the chance we could have a move as small as -12.77%, is 7.326261431744285e-26. This probability is so low that we would never expect an event like this to happen in our lifetime. We have multiple events like this, as illustrated by the minimum and maximum.

Going further with this idea, based on normal distribution z tables, we would expect 99.7% of our data points to be within +/- 3 standard deviations from the mean. Let's determine this for our sample data. First off, we need to find the cut-off values at +/- 3 standard deviations:

In [ ]:
(3 * df.SP500.std()) + df.SP500.mean()

In [ ]:
(-3 * df.SP500.std()) + df.SP500.mean()

The above two calculations would imply that 99.7% of all of our data points should be in between -0.0364 and 0.03699. Since we have 5,031 data points, we would expect about 15 of them to be outside of that range.

In [ ]:
df[(df["SP500"] > 0.03699) | (df["SP500"] < -0.0364)].tail()

In [ ]:
len(df[(df["SP500"] > 0.05) | (df["SP500"] < -0.05)])

Not only do we get 15 values outside of our 3 standard deviation range, but we also get 36 values outside of +/- 5%, though you would almost never expect one of these events over 20 years, given a normal distribution.

In this next video, we go over how to leverage Python in order to test for normality of our data.

In [ ]:
from IPython.display import VimeoVideo

VimeoVideo("706653140", h="47eb01d16b", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1aJ1WcWvEEM5cUnJZm4lDdyOug1j2E9Rs/view?usp=sharing)

All this analysis does is basically prove that we have fat tails in our sample data, which is why the normal distribution is not suitable for modeling stock returns.



## 3. Non-Gaussian Distributions

One potential alternative distribution we could use to forecast stock returns is the Student's t-distribution. This is very similar to a normal distribution except it has heavier tails. Theoretically, this sounds perfect for daily returns based on what we've seen up to this point.

Lets proceed with a visual inspection of the distribution of our data by superimposing the normal distribution on the KDE of S&P500 returns:

In [ ]:
# Sampling from normal distribution
np.random.seed(222)
normal_dist = stats.norm.rvs(
    size=len(df["SP500"]), loc=df["SP500"].mean(), scale=df["SP500"].std()
)

# Creating an additional column in df in order to use the KDE plot functionality of pandas
df["Normal Sample"] = normal_dist

# Plotting the KDE plots
df[["SP500", "Normal Sample"]].plot(kind="kde", xlim=(-0.1, 0.1), figsize=(12, 4))

"""#Using Seaborn to create KDE
plt.figure(figsize = (12,4))
kde = sns.kdeplot(df, fill=True, alpha=.5, linewidth=0).set_xlim(-0.1, 0.1);"""

The SP500 returns seem a lot more leptokurtic. Indeed the excess kurtosis of SP500 is greater than 0:


In [ ]:
df.SP500.kurt()

The tails of SP500 are also fatter than those of a normal distribution:

In [ ]:
# Observing the tails
df[["SP500", "Normal Sample"]].plot(
    kind="kde", xlim=(-0.075, -0.04), ylim=(-1, 2), figsize=(12, 4)
);

At this stage, we will calibrate the parameters of the Student's t-distribution using Maximum Likelihood Estimation (MLE) to align the distribution closely with our observed data.

From the next visual inspection, it's evident that the synthetic data generated from the fitted Student's t-distribution offers a more accurate approximation of our actual data compared to what we got from a normal distribution.


In [ ]:
# Fit the t-distribution using MLE: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fit.html
params = stats.t.fit(df.SP500)

# We plot the fitted distribution against the kde of the data
df["t-Sample *Fitted*"] = stats.t.rvs(*params, size=len(df))
df[["SP500", "t-Sample *Fitted*"]].plot(kind="kde", figsize=(12, 4), xlim=(-0.1, 0.1));

The tails also seem to be better explained by the t-distribution.

In [ ]:
df[["SP500", "t-Sample *Fitted*"]].plot(
    kind="kde", figsize=(12, 4), xlim=(-0.075, -0.04), ylim=(-1, 2)
);

## 4. Conclusion

In this lesson, we focused for the first time on comparing daily stock return data to normal distributions. We also touched on other potential distributions we could use to model this data. In the next module, we will take what we've learned thus far and apply it to a portfolio setting instead of just looking at individual assets in isolation

**References**

- D'Agostino, Ralph B. "An Omnibus Test of Normality for Moderate and Large Size Samples." *Biometrika*, vol. 58, no. 2, 1971, pp. 341–348, https://doi.org/10.1093/biomet/58.2.341.

---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
